Working with DuckDB in Python via ibis

In [1]:
import sqlparse #To make the SQL prettier
import duckdb #Backend
import ibis #Data API

#Extra code to automatically run and return queries
# ibis.options.interactive = True

In [2]:
# Define the database file
db_file = "data/database/example.duckdb"

# Connect to the DuckDB file
conn = ibis.duckdb.connect(database = db_file)

In [3]:
# List tables in the database
tables = conn.list_tables()
print(tables)

['CROP_DATA', 'DAILY_FX', 'FARM_PRICES', 'MONTHLY_FX']


In [4]:
# Access tables using ibis
daily_fx = conn.table("DAILY_FX")
farm_prices = conn.table("FARM_PRICES")

In [5]:
# Show the table structure
print(daily_fx)
print(farm_prices)

DatabaseTable: DAILY_FX
  DFX_ID   float64
  DATE     date
  FXUSDCAD float64
DatabaseTable: FARM_PRICES
  CD_ID        float64
  DATE         date
  CROP_TYPE    string
  GEO          string
  PRICE_PRERMT float64


In [6]:
# Show some table data
print(daily_fx.execute().head(5))
print(farm_prices.execute().head(5))

   DFX_ID        DATE  FXUSDCAD
0     0.0  2017-01-03    1.3435
1     1.0  2017-01-04    1.3315
2     2.0  2017-01-05    1.3244
3     3.0  2017-01-06    1.3214
4     4.0  2017-01-09    1.3240
   CD_ID        DATE CROP_TYPE           GEO  PRICE_PRERMT
0    0.0  1985-01-01    Barley       Alberta        127.39
1    1.0  1985-01-01    Barley  Saskatchewan        121.38
2    2.0  1985-01-01    Canola       Alberta        342.00
3    3.0  1985-01-01    Canola  Saskatchewan        339.82
4    4.0  1985-01-01       Rye       Alberta        100.77


In [7]:
# Join data and print the SQL statement used
#daily_fx.join??
joined_table = daily_fx.join(farm_prices, "DATE", how = "inner")

# Compile the query to SQL (similar to `show_query()` in R)
sql_str = sqlparse.format( #Format the SQL
    (joined_table.
        compile() #Get the SQL
        .replace('"', '')), #Make it friendlier to human eyes
    reindent = True, 
    keyword_case = 'upper'
)

print(sql_str)

SELECT t2.DFX_ID,
       t2.DATE,
       t2.FXUSDCAD,
       t3.CD_ID,
       t3.CROP_TYPE,
       t3.GEO,
       t3.PRICE_PRERMT
FROM DAILY_FX AS t2
INNER JOIN FARM_PRICES AS t3 ON t2.DATE = t3.DATE


In [8]:
# Execute the query and fetch the first 5 rows
print(joined_table.execute().head(5))

   DFX_ID        DATE  FXUSDCAD   CD_ID CROP_TYPE           GEO  PRICE_PRERMT
0    21.0  2017-02-01    1.3074  2331.0    Barley       Alberta        217.95
1    21.0  2017-02-01    1.3074  2332.0    Barley  Saskatchewan        207.77
2    21.0  2017-02-01    1.3074  2333.0    Canola       Alberta        492.53
3    21.0  2017-02-01    1.3074  2334.0    Canola  Saskatchewan        494.45
4    21.0  2017-02-01    1.3074  2335.0       Rye  Saskatchewan        142.36


In [9]:
#Close the connection to the database
conn.disconnect()